In [ ]:
%pip install llama-index llama-index-embeddings-huggingface llama-index-llms-openai load_dotenv spacy 

## Perplexity API Setup

In [ ]:
import requests
import os

from dotenv import load_dotenv
load_dotenv()

url = "https://api.perplexity.ai/chat/completions"

DEFAULT_CONTEXT_TEMPLATE = (
    "Context information is below."
    "\n--------------------\n"
    "{context_str}"
    "\n--------------------\n"
)

def chat_completion(context_str,model,question):
    payload = {
        "model": model,
        "messages": [
            {
                "role": "system",
                "content": DEFAULT_CONTEXT_TEMPLATE.format(context_str=context_str)
            },
            {
                "role": "user",
                "content": question
            }
        ],
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "authorization": f"Bearer {os.getenv('PERPLEXITY_API_KEY')}"
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        json_response = response.json()
        return json_response.get("choices")[0].get("message").get("content")
    elif response.status_code == 429:
        return response.status_code
    elif response.status_code == 400:
        return response.status_code
    else:
        return False

## Index Loading

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding("BAAI/bge-large-en-v1.5")

Repeat form here


In [ ]:
import time
import csv
import signal
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.schema import MetadataMode

top_k_list = [3, 12, 20, 25, 30, 35, 38, 40]


def response_generator(model_name,question_file,index_dir):
    timeout_duration = 10

    vector_index = load_index_from_storage(storage_context=StorageContext.from_defaults(persist_dir=rf'index/{index_dir}'),embed_model=embed_model,llm=None)
    # query_engine = index.as_query_engine()

    with open(question_file, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        csv_data = list(reader)

    class TimeoutException(Exception):
        pass

    def timeout_handler(signum, frame):
        raise TimeoutException("Operation timed out")

    # Loop from 3 to 30 top_k
    try:
        for top_k in top_k_list:
            print(f" ----------------- Details ------------------ ")
            print(f"Model Name: {model_name}")
            print(f"Top K: {top_k}")
            print(f"Number of Questions: {len(csv_data) - 1}")
            print(f"Index: index/{index_dir}")
            print(f"--------------------------------------------- ")

            for row in csv_data:
                index = csv_data.index(row)
                if index == 0:
                    csv_data[index].append(f"{model_name} top_k={top_k}")
                    csv_data[index].append(f"Response time of {model_name} top_k={top_k}")
                    continue

                print(f"-------------------------- Question No : {index} & Top K : {top_k} --------------------------")
                print(f"Question: {row[0]}")
                
                signal.signal(signal.SIGALRM, timeout_handler)
                signal.alarm(timeout_duration)

                try:
                    print(f"Case does not exist")
                    retriever = vector_index.as_retriever(similarity_top_k=top_k)
                    retrieved_nodes = retriever.retrieve(str(row[0]))
                    context_str = "\n\n".join([n.node.get_content(metadata_mode=MetadataMode.LLM).strip() for n in retrieved_nodes])
                    print(f"Vector Response: {context_str}")
                    signal.alarm(0)
                except TimeoutException:
                    print(f"Error: Unable to retrieve vector response (Timeout)")
                    with open(f'csv/{index_dir}-index-response.csv', mode='w', newline='', encoding='utf-8') as file:
                        writer = csv.writer(file)
                        csv_data[index].append("Error : Unable to retrieve vector response (Timeout)")
                        csv_data[index].append(f"Error : Unable to retrieve vector response (Timeout)")
                        writer.writerows(csv_data)

                    continue
                except Exception as e:
                    print(f"Error: {e}")
                    with open(f'csv/{index_dir}-index-response.csv', mode='w', newline='', encoding='utf-8') as file:
                        writer = csv.writer(file)
                        csv_data[index].append("Error : Unable to retrieve vector response")
                        csv_data[index].append(f"Error : Unable to retrieve vector response")
                        writer.writerows(csv_data)

                    continue


                signal.signal(signal.SIGALRM, timeout_handler)
                signal.alarm(timeout_duration)

                try:
                    context_str = f"Context: {context_str}"
                    question_text = f"Question: {row[0]}\n"
                    prompt = f"Using the provided context, answer the following question: \n{question_text}"

                    start_time = time.time()
                    response_llm = chat_completion(model=model_name, question=question_text, context_str=context_str)
                    end_time = time.time()
                    response_time = end_time - start_time
                    print(f"LLM Response: {response_llm}")
                    print(f"Response Time: {response_time:.4f} seconds")

                    # Append the LLM response and the corresponding response time
                    csv_data[index].append(response_llm)
                    csv_data[index].append(f"{response_time:.4f} seconds")

                    signal.alarm(0)
                    with open(f'csv/{index_dir}-index-response.csv', mode='w', newline='', encoding='utf-8') as file:
                        writer = csv.writer(file)
                        writer.writerows(csv_data)

                except TimeoutException:
                    print(f"Error: Unable to retrieve LLM response (Timeout)")
                    with open(f'csv/{index_dir}-index-response.csv', mode='w', newline='', encoding='utf-8') as file:
                        writer = csv.writer(file)
                        csv_data[index].append("Error : Unable to retrieve LLM response (Timeout)")
                        csv_data[index].append(f"Error : Unable to retrieve LLM response (Timeout)")
                        writer.writerows(csv_data)

                    continue

                except Exception as e:
                    print(f"Error: {e}")
                    with open(f'csv/{index_dir}-index-response.csv', mode='w', newline='', encoding='utf-8') as file:
                        writer = csv.writer(file)
                        csv_data[index].append("Error : Unable to retrieve LLM response")
                        csv_data[index].append(f"Error : Unable to retrieve LLM response")
                        writer.writerows(csv_data)
                        
                    continue

    except Exception as e:
        print(f"Error: {e}")

In [ ]:
model_name = 'llama-3.1-8b-instruct' 
question_file = 'csv/legal-questions.csv'
index_dir = 'legal/512-legal' 

response_generator(model_name,question_file,index_dir)